In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.1.post1


In [2]:
!python -V

Python 3.12.2


In [3]:
import pickle
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
year = 2023
month = 3
taxi_type = 'yellow'
output_file = f'{taxi_type}_tripdata_{year:04d}-{month:02d}_predicted.parquet'

In [7]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet')

In [8]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,18.4,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,15.6,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00,11.466667
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.033333


In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q1. Notebook

In [10]:
y_pred.std()

6.247488852238703

### Q2. Preparing the output

In [11]:
#create an artificial ride_id column:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = pd.DataFrame()
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['predicted_duration'] = y_pred

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)


In [12]:
df_result.head()

,ride_id,predicted_duration
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486


In [13]:
!ls -al

total 134192
drwxr-xr-x  8 billehsarkozy  staff       256 Jun 18 18:41 .
drwxr-xr-x  6 billehsarkozy  staff       192 Jun 18 17:38 ..
-rw-r--r--@ 1 billehsarkozy  staff      6148 Jun 18 17:59 .DS_Store
-rw-r--r--  1 billehsarkozy  staff       176 Jun 18 18:25 Pipfile
-rw-r--r--  1 billehsarkozy  staff     17560 Jun 18 18:29 Pipfile.lock
-rw-r--r--@ 1 billehsarkozy  staff      5628 Jun 18 18:31 homework.ipynb
-rw-r--r--@ 1 billehsarkozy  staff     17376 Jun 18 17:38 model.bin
-rw-r--r--  1 billehsarkozy  staff  68641880 Jun 18 18:41 yellow_tripdata_2023-03_predicted.parquet


### Q3. Creating the scoring script

In [ ]:
#pip install ipynb-py-convert
#jupyter nbconvert --to script homework.ipynb

### Q4. Virtual environment

In [ ]:
#install pyenv
curl https://pyenv.run | bash

#copy the below to vi ~/.bash_profile
#pyenv profile
export PYENV_ROOT="$HOME/.pyenv"
[[ -d $PYENV_ROOT/bin ]] && export PATH="$PYENV_ROOT/bin:$PATH"
eval "$(pyenv init -)"

#exit and restart bash 

pipenv install --python 3.10.13
pipenv install scikit-learn==1.5.0 flask 
pipenv shell
PS1="> "
pipenv lock

In [ ]:
#sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c

### Q5. Parametrize the script

In [ ]:
>python homework.py  2023 4
Loading model...
Reading data...
Predicting trip duration...
Predicted mean duration 14.292282936862449
Exporting data...

### Q6. Docker container

docker build -t mlops-zoomcamp-deployment:latest .

In [ ]:
docker run -it mlops-zoomcamp-deployment:latest ./homework.py 2023 5


### Publishing the image to dockerhub

docker build -t mlops-zoomcamp-model:2024-3.10.13-slim .
docker tag mlops-zoomcamp-model:2024-3.10.13-slim agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

docker login --username USERNAME
docker push agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim